In [1]:
import numpy as np
from numpy.linalg import inv, qr, det
import pandas as pd
import math
import re
import matplotlib.pyplot as plt
import nltk

In [82]:
#importing raw data
df=pd.read_excel('2018VAERSDATA.xlsx')
#df2=df.copy
df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,732217.0,2018-01-01,TN,66.00,66.0,NaN,F,NaT,"Constant pain in shoulder, Limited range of mo...",NaN,...,Joint range of motion decreased,20.1,Musculoskeletal pain,20.1,NaN,NaN,NaN,NaN,NaN,NaN
1,732218.0,2018-01-01,NY,0.33,0.0,0.3,F,NaT,"Rash and hives over mouth, neck chest, belly, ...",NaN,...,Ear swelling,20.1,Rash,20.1,Urticaria,20.1,NaN,NaN,NaN,NaN
2,732219.0,2018-01-01,NY,71.00,71.0,NaN,F,NaT,Patient called the pharmacy stating her right ...,NaN,...,Cellulitis,20.1,Injection site erythema,20.1,Injection site pain,20.1,Injection site swelling,20.1,NaN,NaN
3,732220.0,2018-01-01,NJ,80.00,80.0,NaN,F,NaT,"Diffuse rash, warm to the touch surrounding ad...",NaN,...,Rash,20.1,Skin warm,20.1,NaN,NaN,NaN,NaN,NaN,NaN
4,732221.0,2018-01-01,NaN,26.00,26.0,NaN,M,NaT,"Itchy feeling in arm, large red circle around ...",NaN,...,Injection site erythema,20.1,Injection site induration,20.1,Injection site warmth,20.1,Pruritus,20.1,NaN,NaN


list(df)

In [86]:
df=df.fillna(value='nanval')
# removal of rows with incomplete info
# remove ages with with NaN
# remove genders with U, 
# remove numdays with NaN
# remove v_adminby with 'UNK'
# remove Vax_type with 'UNK'
# remove Vax_manu with 
# remove VAX_ROUTE with nan
# remove VAX_SITE wiht nan
# words similar to nan
checkwords=('nanval,''unknown','unk','none','n/a','no')
for index, row in df.iterrows():
    if df.loc[index,'AGE_YRS']== 'nanval':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'SEX'] == 'U':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'NUMDAYS'] == 'nanval':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'V_ADMINBY'] == 'UNK' or df.loc[index,'V_ADMINBY'] == 'OTH':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_TYPE'] == 'UNK':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_MANU'] == 'UNKNOWN MANUFACTURER':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_ROUTE'] == 'nanval' or df.loc[index,'VAX_ROUTE'] == 'UN' or df.loc[index,'VAX_ROUTE'] == 'OT':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_SITE'] == 'nanval' or df.loc[index,'VAX_SITE'] == 'UN' or df.loc[index,'VAX_SITE'] == 'OT':
        df=df.drop(index,axis=0)
        continue
    if row['AGE_YRS'] < (1/365):
        df=df.drop(index,axis=0)
        continue
    if (row['NUMDAYS']/365)>row['AGE_YRS']:
        df=df.drop(index,axis=0)
        continue
        
   

In [87]:
#save trimmed data
df.to_excel('VAE18_V1.xlsx')

In [94]:
# import 2017 data for formating identical to above 2018 data
df=pd.read_excel('2017VAERSDATA.xlsx')
df=df.fillna(value='nanval')
# removal of rows with incomplete info
# remove ages with with NaN
# remove genders with U, 
# remove numdays with NaN
# remove v_adminby with 'UNK'
# remove Vax_type with 'UNK'
# remove Vax_manu with 
# remove VAX_ROUTE with nan
# remove VAX_SITE wiht nan
checkwords=('nanval,''unknown','unk','none','n/a','no')
for index, row in df.iterrows():
    if df.loc[index,'AGE_YRS']== 'nanval':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'SEX'] == 'U':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'NUMDAYS'] == 'nanval':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'V_ADMINBY'] == 'UNK' or df.loc[index,'V_ADMINBY'] == 'OTH':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_TYPE'] == 'UNK':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_MANU'] == 'UNKNOWN MANUFACTURER':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_ROUTE'] == 'nanval' or df.loc[index,'VAX_ROUTE'] == 'UN' or df.loc[index,'VAX_ROUTE'] == 'OT':
        df=df.drop(index,axis=0)
        continue
    if df.loc[index,'VAX_SITE'] == 'nanval' or df.loc[index,'VAX_SITE'] == 'UN' or df.loc[index,'VAX_SITE'] == 'OT':
        df=df.drop(index,axis=0)
        continue
    if row['AGE_YRS'] < (1/365):
        df=df.drop(index,axis=0)
        continue
    if (row['NUMDAYS']/365)>row['AGE_YRS']:
        df=df.drop(index,axis=0)

df.to_excel('VAE17_V1.xlsx')

In [102]:
# opening both sets of data for merging
df18=pd.read_excel('VAE18_V1.xlsx')
df17=pd.read_excel('VAE17_V1.xlsx')
#df2=df.copy
df17.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
5,676022,2017-01-01,OK,35.0,35,nanval,F,2017-01-01 00:00:00,Tingling in feet.,nanval,...,Pain in extremity,19.1,Rash,19.1,nanval,nanval,nanval,nanval,nanval,nanval
12,676029,2017-01-01,TX,67.0,67,nanval,M,2017-01-01 00:00:00,Dull ache area of shot. Severe pain when musc...,nanval,...,Nuclear magnetic resonance imaging,19.1,Respiration abnormal,19.1,Respiratory disorder,19.1,Respiratory failure,19.1,Resuscitation,19.1
47,676090,2017-01-02,OR,52.0,52,nanval,F,2017-01-02 00:00:00,11/4/2016 Onset of Pain at injection site. 11/...,nanval,...,Allergy to vaccine,19.1,nanval,nanval,nanval,nanval,nanval,nanval,nanval,nanval
60,676104,2017-01-02,ID,11.0,11,nanval,F,2017-01-02 00:00:00,Patient woke up complaining her hand wouldn't ...,nanval,...,Incorrect product storage,19.1,No adverse event,19.1,nanval,nanval,nanval,nanval,nanval,nanval
69,676291,2017-01-02,MI,16.0,16,nanval,M,2017-01-02 00:00:00,"102 Temp, weakness, unable to bare weight or w...",nanval,...,Ear pain,19.1,Neck pain,19.1,Tenderness,19.1,nanval,nanval,nanval,nanval


In [105]:
#m merging 2017 and 2018 data
df=pd.concat([df17, df18], ignore_index=True)
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,676022,2017-01-01,OK,35.00,35,nanval,F,2017-01-01 00:00:00,Tingling in feet.,nanval,...,Pain in extremity,19.1,Rash,19.1,nanval,nanval,nanval,nanval,nanval,nanval
1,676029,2017-01-01,TX,67.00,67,nanval,M,2017-01-01 00:00:00,Dull ache area of shot. Severe pain when musc...,nanval,...,Nuclear magnetic resonance imaging,19.1,Respiration abnormal,19.1,Respiratory disorder,19.1,Respiratory failure,19.1,Resuscitation,19.1
2,676090,2017-01-02,OR,52.00,52,nanval,F,2017-01-02 00:00:00,11/4/2016 Onset of Pain at injection site. 11/...,nanval,...,Allergy to vaccine,19.1,nanval,nanval,nanval,nanval,nanval,nanval,nanval,nanval
3,676104,2017-01-02,ID,11.00,11,nanval,F,2017-01-02 00:00:00,Patient woke up complaining her hand wouldn't ...,nanval,...,Incorrect product storage,19.1,No adverse event,19.1,nanval,nanval,nanval,nanval,nanval,nanval
4,676291,2017-01-02,MI,16.00,16,nanval,M,2017-01-02 00:00:00,"102 Temp, weakness, unable to bare weight or w...",nanval,...,Ear pain,19.1,Neck pain,19.1,Tenderness,19.1,nanval,nanval,nanval,nanval
5,676304,2017-01-02,TX,11.00,11,nanval,M,2016-12-29 00:00:00,Pt allegedly fainted for less than 5 seconds. ...,nanval,...,Incorrect product storage,19.1,No adverse event,19.1,nanval,nanval,nanval,nanval,nanval,nanval
6,676169,2017-01-03,OH,0.50,0,0.5,M,2017-01-03 00:00:00,Patient broke out in hives. First hives starte...,nanval,...,Incorrect product storage,19.1,No adverse event,19.1,nanval,nanval,nanval,nanval,nanval,nanval
7,676247,2017-01-03,MA,36.00,36,nanval,M,2017-01-03 00:00:00,Felt fine in office after administration of TD...,nanval,...,Injection site erythema,19.1,Injection site pain,19.1,Injection site swelling,19.1,nanval,nanval,nanval,nanval
8,676301,2017-01-03,TN,10.00,10,nanval,M,2016-12-29 00:00:00,12-29-16 Health center called an stated the pa...,nanval,...,Circulatory collapse,19.1,Resuscitation,19.1,Shock,19.1,nanval,nanval,nanval,nanval
9,676302,2017-01-03,ME,17.00,17,nanval,M,2017-01-03 00:00:00,Pt. was accidently given wrong dose of influen...,nanval,...,Eye pain,19.1,Injection site inflammation,19.1,Injection site swelling,19.1,Iritis,19.1,Malaise,19.1


In [108]:
# saving new 2017+2018 data
df.to_excel('VAE1817_V1.xlsx')

In [98]:
# Symptom Severity function
# levels of severity are based on triage standards
def symp_sev(df,i,cols,numsym):
    rank=0
    severe = 0
    # list of max level severity symptoms
    sevcheck = ['seizure','unconscious','heart attack','hypoperfusion','weak pulse','cyanosis'
                ,'unable to speak','tachycardia','bradycardia','myocardial infarction','paroxysm','collapse','spasm']
    # list of level 4 symptoms
    lvl4 = ['stridor','difficulty breathing','confusion','memory','loss of orientation','paralysis','laryngospasm'
            ,'apnea','hyperglycemia']
    # list of level 3 symptoms
    lvl3 = ['dyspnea','shortness of breath','tachypnea','aesthenia','vision blurred','blurry vision','difficulty urinating'
           ,'asthma','epilepsy']
    # list of level 2 symptoms
    lvl2 = ['diarrhea','nausea','vomit','weakness','blister','rash','constipation','gastritis','pyrexia'
            ,'bronchospasm','arrhythmia','edema']
    # list of level 1 symptoms
    lvl1 = ['warm','urticaria','malaise','myalgia','paraethesia','swelling','itch','puritus','fatigue']
    # cycle through symptom columns to get composite score
    for symps in cols:
        # scan for max severity
        # if max severity symptom is found, break scan and set severity to max
        if any(cword in str(df.loc[index,symps]).lower() for cword in sevcheck):
            severe = 1
            break
        # check for pain related symptoms, rank based on existance of quality words
        if 'pain' in str(df.loc[index,symps]).lower():
            if 'severe' in str(df.loc[index,symps]).lower():
                rank+=4
            elif 'moderate' in str(df.loc[index,symps]).lower():
                rank+=3
            else:
                rank+=2
        # check for bleed symptoms, rank based on quality words existances
        elif 'bleed' in str(df.loc[index,symps]).lower():
            if 'internal' in str(df.loc[index,symps]).lower():
                rank+=4
            elif 'severe' in str(df.loc[index,symps]).lower():
                rank+=4
            elif 'moderate' in str(df.loc[index,symps]).lower():
                rank+=2
            else:
                rank+=1
        # Run through above severity level words in descending severity
        else:
            if any(cword in str(df.loc[index,symps]).lower() for cword in lvl4):
                rank+=4
            elif any(cword in str(df.loc[index,symps]).lower() for cword in lvl3):
                rank+=3
            elif any(cword in str(df.loc[index,symps]).lower() for cword in lvl2):
                rank+=2
            elif any(cword in str(df.loc[index,symps]).lower() for cword in lvl1):
                rank+=1
    #print(rank)
    if severe == 1:
        return 5
    # standard severity
    else:
        return rank/numsym
                
    

In [109]:
df=pd.read_excel('VAE1817_V1.xlsx')

index = 1
if df.loc[index,'SYMPTOM1'] != 'nanval':
    if df.loc[index,'SYMPTOM2'] != 'nanval':
        if df.loc[index,'SYMPTOM3'] != 'nanval':
            if df.loc[index,'SYMPTOM4'] != 'nanval':
                if df.loc[index,'SYMPTOM5'] != 'nanval':
                    df.loc[index,'NUM_SYMPS'] = 5
                else:
                    df.loc[index,'NUM_SYMPS'] = 4
            else:
                df.loc[index,'NUM_SYMPS'] = 3
        else:
            df.loc[index,'NUM_SYMPS'] = 2
    else:
        df.loc[index,'NUM_SYMPS'] = 1
else:
    df.loc[index,'NUM_SYMPS'] = 0
df.loc[index,'SYMP_SEV']=symp_sev(df,index,['SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5'],df.loc[index,'NUM_SYMPS'])

df

df.sort_values('AGE_YRS',ascending=False)

In [110]:
#df=pd.read_excel('VAE1817_V1.xlsx')
#Feature Formation
checkwords=('nanval','unknown','unk','none','n/a','no ')
for index, row in df.iterrows():
    #Severity calculation 
    # Death is severity 3
    # Disability is severity 2
    # Hospitalization or L_Threat or ER_visit are severity 1
    # Otherwise just severity 0
    #print(df.loc[index,'DIED'])
    if any(df.loc[index,'DIED']) == 'Y':
        df.loc[index,'SEVERITY'] = 3
    elif df.loc[index,'DISABLE'] == 'Y':
        df.loc[index,'SEVERITY'] = 2
    elif df.loc[index,'HOSPITAL'] == 'Y' or df.loc[index,'L_THREAT']== 'Y' or df.loc[index,'ER_VISIT'] == 'Y':
        df.loc[index,'SEVERITY'] = 1
    else:
        df.loc[index,'SEVERITY'] = 0
    
    #format Sex to binary - set M/F to 0=male 1=female
    if df.loc[index,'SEX']=='M':
        df.loc[index,'M/F'] = 0
    elif df.loc[index,'SEX'] == 'F':
        df.loc[index,'M/F'] = 1
    #format 'BIRTH_DEFFECT'
    # Check for Y values and set 1=Yes and 0=no
    if df.loc[index,'BIRTH_DEFECT'] == 'Y':
            df.loc[index,'BIRTHDEF_Y/N'] = 1
    else:
            df.loc[index,'BIRTHDEF_Y/N'] = 0
    # Number of Symptoms feature
    # Check for entries in the numbered symptons features
    # Presence of a non NaN entry means a symptom was recorded
    if df.loc[index,'SYMPTOM1'] != 'nanval':
        if df.loc[index,'SYMPTOM2'] != 'nanval':
            if df.loc[index,'SYMPTOM3'] != 'nanval':
                if df.loc[index,'SYMPTOM4'] != 'nanval':
                    if df.loc[index,'SYMPTOM5'] != 'nanval':
                        df.loc[index,'NUM_SYMPS'] = 5
                    else:
                        df.loc[index,'NUM_SYMPS'] = 4
                else:
                    df.loc[index,'NUM_SYMPS'] = 3
            else:
                df.loc[index,'NUM_SYMPS'] = 2
        else:
            df.loc[index,'NUM_SYMPS'] = 1
    else:
        df.loc[index,'NUM_SYMPS'] = 0
    df.loc[index,'SYMP_SEV']=symp_sev(df,index,['SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5']
                                      ,df.loc[index,'NUM_SYMPS'])
    # formatting for features with NLP requirements
    # checkwords based on manual curation of common terms indicating should be NaN value
    #for cword in checkwords:
        #format 'OTHER_MEDS'
        # Binary 1=yes other meds, 0 = no other meds
    if any(cword in str(df.loc[index,'OTHER_MEDS']).lower() for cword in checkwords):
        df.loc[index,'MEDS_Y/N'] = 0
    else:
        df.loc[index,'MEDS_Y/N'] = 1         
    #Format Dose series info
    # Fix unknown values to 1 assuming singular dose in that series
    ## may revisit and just remove these values
    if any(cword in str(df.loc[index,'VAX_DOSE_SERIES']).lower() for cword in checkwords):
        df.loc[index,'VAX_DOSE_SERIES'] = 1
    elif row['VAX_DOSE_SERIES'] == '7+':
        df.loc[index,'VAX_DOSE_SERIES'] = 7
    #Format CUR_ILL
    # simple 1 for yes current illness and 0 for no current illness
    if any(cword in str(df.loc[index,'CUR_ILL']).lower() for cword in checkwords):
        df.loc[index,'CILL_Y/N'] = 0
    else:
        df.loc[index,'CILL_Y/N'] = 1
    #format 'HISTORY'
    # Format for 1 pervious history of AE, 0 for no history
    if any(cword in str(df.loc[index,'HISTORY']).lower() for cword in checkwords):
        df.loc[index,'HIST_Y/N'] = 0
    else:
        df.loc[index,'HIST_Y/N'] = 1
    #format 'PRIOR_VAX'
    # 1 = yes prior VE 0 = no prior VE
    if any(cword in str(df.loc[index,'PRIOR_VAX']).lower() for cword in checkwords):
        df.loc[index,'PVAX_Y/N'] = 0
    else:
        df.loc[index,'PVAX_Y/N'] = 1

    #format 'ALLERGIES'
    # check for listing of allergies - relevancy is not checked
    if any(cword in str(df.loc[index,'ALLERGIES']).lower() for cword in checkwords):
        df.loc[index,'ALLERG_Y/N'] = 0
    else:
        df.loc[index,'ALLERG_Y/N'] = 1

            
#Column Dropping
# List of columns to drop
dcolumns = ('SEX','RECVDATE',"STATE","CAGE_YR","CAGE_MO",'RPT_DATE','SYMPTOM_TEXT','DIED','DATEDIED','L_THREAT','ER_VISIT','HOSPITAL','HOSPDAYS'
            ,'X_STAY','DISABLE','RECOVD','VAX_DATE','ONSET_DATE','LAB_DATA','V_FUNDBY','SPLTTYPE','FORM_VERS','TODAYS_DATE'
           ,'OFC_VISIT','ER_ED_VISIT','VAX_LOT','VAX_NAME','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX',
            'BIRTH_DEFECT','ALLERGIES','SYMPTOM1','SYMPTOMVERSION1','SYMPTOM2','SYMPTOMVERSION2','SYMPTOM3','SYMPTOMVERSION3'
           ,'SYMPTOM4','SYMPTOMVERSION4','SYMPTOM5','SYMPTOMVERSION5')
for x in dcolumns:
    df=df.drop(x,axis=1)
df.to_excel('VAE1817_V2.xlsx')

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
Y
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanv

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
Y
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanv

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

...

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
Y
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanv

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
Y
nanval
nanval
nanval
nanval
nanval
nanval
nanval
Y
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
na

nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval
nanval

In [5]:
df=pd.read_excel('VAE1817_V2.xlsx')
df

,VAERS_ID,AGE_YRS,NUMDAYS,V_ADMINBY,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,SEVERITY,M/F,BIRTHDEF_Y/N,NUM_SYMPS,SYMP_SEV,MEDS_Y/N,CILL_Y/N,HIST_Y/N,PVAX_Y/N,ALLERG_Y/N
0,676022,35.00,18,PVT,TDAP,SANOFI PASTEUR,1,IM,RA,1,1,0,2,2.000000,0,0,0,0,0
1,676029,67.00,4,PVT,MMR,MERCK & CO. INC.,1,IM,LL,0,0,0,5,0.000000,0,1,0,0,0
2,676090,52.00,3,PVT,FLU4,GLAXOSMITHKLINE BIOLOGICALS,2,IM,RA,2,1,0,1,0.000000,0,0,1,0,0
3,676104,11.00,52,PUB,FLU3,SANOFI PASTEUR,1,IM,LA,1,1,0,2,0.000000,0,0,0,0,0
4,676291,16.00,0,PVT,FLU3,NOVARTIS VACCINES AND DIAGNOSTICS,1,IM,RA,1,0,0,3,1.333333,1,1,1,0,0
5,676304,11.00,0,PVT,PPV,MERCK & CO. INC.,1,IM,LA,0,0,0,2,0.000000,1,0,0,0,0
6,676169,0.50,2,PUB,RV5,MERCK & CO. INC.,1,PO,MO,1,0,0,2,0.000000,0,0,0,1,0
7,676247,36.00,0,PVT,HIBV,SANOFI PASTEUR,3,IM,RL,0,0,0,3,1.000000,0,1,1,0,0
8,676301,10.00,2,PUB,TDAP,SANOFI PASTEUR,1,IM,LA,0,0,0,3,5.000000,0,0,0,0,0
9,676302,17.00,0,PVT,HEP,GLAXOSMITHKLINE BIOLOGICALS,1,IM,RL,0,0,0,5,0.800000,0,0,0,0,0


In [6]:
# hot encoding categorical data
# V_ADMINBY/VAX_TYPE/VAX_MANU/VAX_ROUTE/VAX_SITE
df=df.drop('V_ADMINBY',axis=1)

one_hot2=pd.get_dummies(df['VAX_TYPE'])
df=df.join(one_hot2)
df=df.drop('VAX_TYPE',axis=1)

one_hot3=pd.get_dummies(df['VAX_MANU'])
df=df.join(one_hot3)
df=df.drop('VAX_MANU',axis=1)

one_hot4=pd.get_dummies(df['VAX_ROUTE'])
df=df.join(one_hot4)
df=df.drop('VAX_ROUTE',axis=1)

one_hot5=pd.get_dummies(df['VAX_SITE'])
df=df.join(one_hot5)
df=df.drop('VAX_SITE',axis=1)

df=df[[c for c in df if c not in ['SEVERITY']] + ['SEVERITY']]
df.to_excel('VAE1817_V3.xlsx')

In [10]:
df=pd.read_excel('VAE1817_V3.xlsx')
df

,VAERS_ID,AGE_YRS,NUMDAYS,VAX_DOSE_SERIES,M/F,BIRTHDEF_Y/N,NUM_SYMPS,SYMP_SEV,MEDS_Y/N,CILL_Y/N,...,AR,GM,LA,LG,LL,MO,NS,RA,RL,SEVERITY
0,676022,35.00,18,1,1,0,2,2.000000,0,0,...,0,0,0,0,0,0,0,1,0,1
1,676029,67.00,4,1,0,0,5,0.000000,0,1,...,0,0,0,0,1,0,0,0,0,0
2,676090,52.00,3,2,1,0,1,0.000000,0,0,...,0,0,0,0,0,0,0,1,0,2
3,676104,11.00,52,1,1,0,2,0.000000,0,0,...,0,0,1,0,0,0,0,0,0,1
4,676291,16.00,0,1,0,0,3,1.333333,1,1,...,0,0,0,0,0,0,0,1,0,1
5,676304,11.00,0,1,0,0,2,0.000000,1,0,...,0,0,1,0,0,0,0,0,0,0
6,676169,0.50,2,1,0,0,2,0.000000,0,0,...,0,0,0,0,0,1,0,0,0,1
7,676247,36.00,0,3,0,0,3,1.000000,0,1,...,0,0,0,0,0,0,0,0,1,0
8,676301,10.00,2,1,0,0,3,5.000000,0,0,...,0,0,1,0,0,0,0,0,0,0
9,676302,17.00,0,1,0,0,5,0.800000,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# binarize severity into 0 and 1
df.loc[df['SEVERITY'] > 0, 'SEVERITY'] = 1
df=df.drop('VAERS_ID',axis=1)
df.to_excel('VAE1817_BSEV_F1.xlsx')

In [12]:
# split data into younger portion
df=pd.read_excel('VAE1817_BSEV_F1.xlsx')
for index, row in df.iterrows():
    if row['AGE_YRS'] > 55.5:
        df=df.drop(index,axis=0)
        continue
df.to_excel('VAE1817_BSEV_F_Young1.xlsx')